# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

* cargar datos
* extraer una muestra
* hacer parseo
* tipos de datos
* cargar todos los datos
* distribuirlos si el espacio no es suficiente
* mismas columnas que el fichero original
* validar dominio (estructura de los datos) summary
* analisis descriptivo
* generamos variables
* estudiar el balance de la variable objetivo
* dividimos en train test
* creamos modelo
* testeamos el modelo


## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

El problema más común que he encontrado es cuando los datos son cargados al sistema manualmente. Esto provoca que haya muchos 'typos' en la información por ejemplo: Madrid,Mmadrid,Maadrid,etc.
Para solucionar esto podríamos usar expresiones regulares que limpien este tipo de datos y dejen un registro único.


Otro problema es la cantidad de tablas que a veces tienen las empresas, ya que es complicado hasta para los
ingenieros en sistemas saber donde está la información que necesitas. O por no buscar te dan lo primero que encuentran lo cual crea problemas a la hroa de realizar modelos.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

Primero importamos las librerias y funciones necesarias.

In [1]:
import re
import datetime
import os
import time

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [2]:
sc=SparkContext('local','Simple App')
sqlContext = SQLContext(sc)

Cargamos los datos y eliminamos los títulos

In [114]:
datos = sc.textFile('partidos.csv').map(lambda x:x.split(":"))
header = datos.first()
datos = datos.filter(lambda x : x != header)

Vemos que se haya cargado correctamente el RDD.

In [115]:
datos.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

Observamos que hubo un ligero problema al descomponer la hora, pero esto lo arreglaremos adelante.

Creamos dos funciones que nos ayudaran a manjear los datos.
La primera para la fecha y la segunda para saber el día de la semana.

In [116]:
def formatofecha(x):
    x=time.strptime(x,'%a, %b %d, %Y"%I:%M %p"')
    return datetime.datetime(x[0],x[1],x[2],x[3],x[4])

In [117]:
def semana(x):
    x=time.strptime(x,'%a, %b %d, %Y')
    return x[6]

Damos formato a los datos

Observamos como en la primer columna junto 3 elementos del RDD para solucionar el problema que surgió a la hora de leer los datos

In [144]:
row_data = datos.map(lambda p: Row(
    fecha=formatofecha(p[0] + p[1] + ':' + p[2] ), 
    diasemana=semana(p[0]),
    equipolocal=p[5],
    equipovisitante=p[3],
    ptslocal=int(p[6]),
    ptsvisitantes=int(p[4]),
    )
)

Contamos solo para asegurarnos de no haber perdido datos

In [129]:
row_data.count()

12897

Creamos la tabla para poder utilizar sqlcontext

In [145]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("basket")

In [146]:
sqlContext.sql("""
    SELECT * from basket limit 10
""").show()

+---------+--------------------+--------------------+--------------------+--------+-------------+
|diasemana|         equipolocal|     equipovisitante|               fecha|ptslocal|ptsvisitantes|
+---------+--------------------+--------------------+--------------------+--------+-------------+
|        1|Golden State Warr...|           Utah Jazz|2007-10-30 19:30:...|      96|          117|
|        1|  Los Angeles Lakers|     Houston Rockets|2007-10-30 19:30:...|      93|           95|
|        1|   San Antonio Spurs|Portland Trail Bl...|2007-10-30 19:00:...|     106|           97|
|        2| Cleveland Cavaliers|    Dallas Mavericks|2007-10-31 20:00:...|      74|           92|
|        2|      Denver Nuggets| Seattle SuperSonics|2007-10-31 20:30:...|     120|          103|
|        2|      Indiana Pacers|  Washington Wizards|2007-10-31 19:00:...|     119|          110|
|        2|   Memphis Grizzlies|   San Antonio Spurs|2007-10-31 19:00:...|     101|          104|
|        2|     New 

## Quinta tarea: Media de la diferencia de puntos por año

Primero restamos los puntos del local menos los puntos del visitante en valor absoluto y aplicamos la media y agrupamos por año.

In [147]:
sqlContext.sql("""
    SELECT avg(abs(ptslocal-ptsvisitantes)) as Promedio,year(fecha) as Year
    from basket group by year(fecha)
    order by year(fecha)
    limit 10
""").show()

+------------------+----+
|          Promedio|Year|
+------------------+----+
|11.096491228070175|2007|
|11.543543543543544|2008|
|11.090425531914894|2009|
| 10.86903860711582|2010|
|10.661016949152541|2011|
|10.845318860244234|2012|
|11.071752265861027|2013|
|  10.9047976011994|2014|
|11.159969673995452|2015|
|11.550637659414853|2016|
+------------------+----+



## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

Esta pregunta la responderé al final ya que me ayudaré de una tabla que cree en la pregunta 9

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

Primero contamos las victorias de cleveland como local

In [149]:
sqlContext.sql("""
    SELECT count(*) as Victoriaslocal from basket 
    where equipolocal = "Cleveland Cavaliers" and month(fecha)=1 and ptslocal>ptsvisitantes
""").show()

+--------------+
|Victoriaslocal|
+--------------+
|            42|
+--------------+



Ahora contamos sus victorias como visitante

In [150]:
sqlContext.sql("""
    SELECT count(*) as Victoriasvisitante 
    from basket 
    where equipovisitante = 'Cleveland Cavaliers' and month(fecha)=1 and ptslocal<ptsvisitantes
""").show()

+------------------+
|Victoriasvisitante|
+------------------+
|                41|
+------------------+



Victorias de Cleveland en el mes de Enero

In [151]:
42+41

83

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

Haremos algo similar a lo de arriba, pero sin la condición del mes de Enero

Local

In [152]:
sqlContext.sql("""
    SELECT count(*) as VictoriasLoc 
    from basket 
    where equipolocal='Golden State Warriors' and ptslocal>ptsvisitantes
""").show()

+------------+
|VictoriasLoc|
+------------+
|         308|
+------------+



Visitante

In [153]:
sqlContext.sql("""
    SELECT count(*) as VictoriasVis 
    from basket 
    where equipovisitante='Golden State Warriors' and ptslocal<ptsvisitantes
""").show()

+------------+
|VictoriasVis|
+------------+
|         215|
+------------+



Los Warriors son mejores en casa

## Novena pregunta: Equipo que ha quedado primerio en victorias más año. (si es que hay alguno que más)

Para empezar crearé dos tablas nuevas:
* Una contiene las victorias y partidos como local
* victorias y partidos como visitante

In [154]:
TablaLoc=sqlContext.sql("""
select equipolocal as Equipo,year(fecha) as year,sum(if(ptsvisitantes<ptslocal,1,0)) as VictoriasLoc, sum(1) as PartidosLoc 
from basket 
group by equipolocal, year(fecha)
    
""")

In [155]:
TablaVis=sqlContext.sql("""
select equipovisitante as Equipo,year(fecha) as year,sum(if(ptsvisitantes>ptslocal,1,0)) as VictoriasVis, sum(1) as PartidosVis 
from basket 
group by equipovisitante, year(fecha)
    
""")

Unimos las tablas para obtener las victorias totales y partidos totales por año

In [156]:
tablaTot=TablaLoc.join(TablaVis,on=['Equipo','year'])

In [157]:
tablaTot.show(3)

+---------------+----+------------+-----------+------------+-----------+
|         Equipo|year|VictoriasLoc|PartidosLoc|VictoriasVis|PartidosVis|
+---------------+----+------------+-----------+------------+-----------+
|New York Knicks|2010|          19|         39|          16|         43|
|  Orlando Magic|2009|          40|         53|          30|         52|
|  Orlando Magic|2017|           9|         23|           5|         25|
+---------------+----+------------+-----------+------------+-----------+
only showing top 3 rows



Registramos la tabla para poder usarlo con sqlcontext

In [158]:
sqlContext.registerDataFrameAsTable(tablaTot,"basket2")

Con sql sumamos partidos y victorias para crear la tabla final

In [159]:
tabla2=sqlContext.sql("""
    select Equipo,VictoriasLoc+VictoriasVis as Victorias,
    PartidosLoc+PartidosVis as Partidos,
    year
    from basket2
""")

In [160]:
sqlContext.registerDataFrameAsTable(tabla2,"basket3")

In [161]:
sqlContext.sql("""
select * from basket3 limit 5
""").show()

+--------------------+---------+--------+----+
|              Equipo|Victorias|Partidos|year|
+--------------------+---------+--------+----+
|     New York Knicks|       35|      82|2010|
|       Orlando Magic|       70|     105|2009|
|       Orlando Magic|       14|      48|2017|
|Minnesota Timberw...|       30|      81|2014|
|    Dallas Mavericks|       47|      86|2015|
+--------------------+---------+--------+----+



Ahora extraemos los equipos que más ganaron por año y creamos una nueva tabla

In [162]:
Victoriasyear=sqlContext.sql("""
select f.equipo, f.year, f.victorias
from (
   select year, max(victorias) as maximo
   from basket3 group by year
) as x inner join basket3 as f on f.year = x.year and f.victorias = x.maximo
""")

In [163]:
sqlContext.registerDataFrameAsTable(Victoriasyear,"Victorias")

Y ahora contamos que equipo aparece más en la tabla creada

In [164]:
sqlContext.sql("""
select count(*) as Conteo,Equipo from victorias group by equipo order by Conteo desc
""").show()

+------+--------------------+
|Conteo|              Equipo|
+------+--------------------+
|     3|Golden State Warr...|
|     2|      Boston Celtics|
|     2|  Los Angeles Lakers|
|     2|   San Antonio Spurs|
|     1|       Chicago Bulls|
|     1|          Miami Heat|
+------+--------------------+



Golden State Warriors que ha quedado más veces primero por victorias por año

(Este proceso no fue muy eficiente)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

Esta expresion busca cuatro secuencias de digitos que tiene un espacio en blanco entre cada una de ellas o cualquier secuencia de caracteres seguido de un signo @ seguido por una secuencia de caracteres seguido por un punto y seguido por otra cadenada de caracteres.

(\d+\s\d+\s\d+\s\d+)|(\w+@\w+\.\w+)

## Sexta tarea: ¿Han jugado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

Con la tabla creada que contiene el número de partidos por equipo solo hace falta extraer los números distintos de partidos por año y ver si hay alguno en que no coincidan

In [165]:
sqlContext.sql("""
select distinct(partidos),year 
from basket3 
order by year
""").show()

+--------+----+
|partidos|year|
+--------+----+
|      32|2007|
|      29|2007|
|      33|2007|
|      28|2007|
|      30|2007|
|      31|2007|
|     101|2008|
|      86|2008|
|      89|2008|
|      90|2008|
|      85|2008|
|      91|2008|
|      51|2008|
|      94|2008|
|      88|2008|
|     112|2008|
|      81|2008|
|      83|2008|
|      92|2008|
|      95|2008|
+--------+----+
only showing top 20 rows



Observamos que sí hay equipos que jugaron más partidos que otros en el mismo año.
Esto puede deberse a las últimas fases de los torneos, ya que como son eliminatorias no todos los equipos juegan y evidentemente en la final sólo juegan 2 equipos por lo que estos tendran un mayor número de juegos.

In [169]:
sc.stop()